# Extract Data for Financial Companies

Gather data of: 

* Stock data from January 2017 to March 2022, including the market capitalization of each company at the end of each quarter.

* Calculation of all applicable financial ratios, such as:
  - Return on equity (ROE): measures the profitability of the company's shareholders' equity
  - Return on assets (ROA): measures the profitability of the company's assets
  - Capital adequacy ratio (CAR): measures the bank's capital against its risk-weighted assets
  - Net interest margin (NIM): measures the profitability of the company's lending activities
  - Non-performing loans (NPL) ratio: measures the percentage of loans that are not being paid back on time
  
* Basic company information, including their Global Industry Classification Standard (GICS) code and sector of activity, as well as their headquarters' location (latitude, longitude, county, city, state, zip code, and number of employees), and the location of any branches, if applicable

In [4]:
# Set working directory and import libraries 
import os
import sys
import time 
import pandas as pd
import requests.exceptions
import json


# Get the current directory
current_dir = os.getcwd()

# Get the parent directory
parent_dir = os.path.dirname(current_dir)

# Add the parent directory to the system path
sys.path.append(parent_dir)

# Import functions created to extract data
from src.data.stock_data import StockData

## Financial Companies

In [ ]:
# Import data with all companies for which we need to gather data
financial_sector_companies = pd.read_excel("../data/raw/financial_sector_companies.xlsx")

financial_sector_companies

## Retrieve Data

In [ ]:
# Define the maximum number of retries
max_retries = 3

# Iterate over the symbols
for symbol in financial_sector_companies.Symbol:
    print(symbol)
    retries = 0
    while retries < max_retries:
        try:
            time.sleep(5)
            data = StockData(symbol)
            data.scrape_financial_statements()
            data.retrieve_profile_stock_price(start="2017-01-01", end="2022-03-31")
            data.save_to_json()
            break  # If everything is successful, break the loop and move to the next symbol
        except requests.exceptions.RequestException as e:
            print(f"A connection error occurred: {e}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries}) in 5 minutes...")
            time.sleep(300)
        except Exception as e:
            print(f"An error occurred: {e}")
            break  # Move to the next symbol for other types of exceptions
    if retries == max_retries:
        print(f"Max retries reached for symbol {symbol}. Moving to the next symbol...")

## Example of retrieved data: JPM

In [7]:
with open("../data/raw/json_data/JPM.json", 'r') as f:
    data = json.load(f)
    
data.keys()

dict_keys(['income_statement', 'balance_sheet', 'cash_flow_statement', 'ratios', 'stock_price', 'profile'])

### Profile

In [9]:
data["profile"]

{'address1': '383 Madison Avenue',
 'city': 'New York',
 'state': 'NY',
 'zip': '10179',
 'country': 'United States',
 'phone': '212 270 6000',
 'website': 'https://www.jpmorganchase.com',
 'industry': 'Banks—Diversified',
 'industryDisp': 'Banks—Diversified',
 'sector': 'Financial Services',
 'longBusinessSummary': 'JPMorgan Chase & Co. operates as a financial services company worldwide. It operates through four segments: Consumer & Community Banking (CCB), Corporate & Investment Bank (CIB), Commercial Banking (CB), and Asset & Wealth Management (AWM). The CCB segment offers deposit, investment and lending products, cash management, and payments and services to consumers and small businesses; mortgage origination and servicing activities; residential mortgages and home equity loans; and credit cards, auto loans, leases, and travel services. The CIB segment provides investment banking products and services, including corporate strategy and structure advisory, and equity and debt market

#### Financial Statements (Quarterly and Annual)

In [12]:
# Income statement
pd.DataFrame(data["income_statement"]["quarterly"])

,Metrics,2023-03-31,2022-12-31,2022-09-30,2022-06-30,2022-03-31,2021-12-31,2021-09-30,2021-06-30,2021-03-31,...,2015-06-30,2015-03-31,2014-12-31,2014-09-30,2014-06-30,2014-03-31,2013-12-31,2013-09-30,2013-06-30,+70 Quarters
1,Revenue,"38,349","34,547","32,716","30,715","30,717","29,257","29,647","30,479","32,266",...,"23,812","24,066","22,750","24,469","24,678","23,215","23,917","23,117","25,211",Upgrade
2,Revenue Growth (YoY),24.85%,18.08%,10.35%,0.77%,-4.80%,-0.27%,1.34%,-7.85%,14.07%,...,-3.51%,3.67%,-4.88%,5.85%,-2.11%,-7.59%,1.12%,-8.07%,13.67%,Upgrade
3,Gross Profit,"38,349","34,547","32,716","30,715","30,717","29,257","29,647","30,479","32,266",...,"23,812","24,066","22,750","24,469","24,678","23,215","23,917","23,117","25,211",Upgrade
4,"Selling, General & Admin","18,468","17,281","17,565","17,156","17,773","16,378","15,904","16,403","17,189",...,"12,526","12,692","12,167","12,791","12,598","12,572","12,223","12,113","12,763",Upgrade
5,Other Operating Expenses,"1,639","1,741","1,613","1,593","1,418","1,510","1,159","1,264","1,536",...,"1,974","2,191","3,242","3,007","2,833","2,064","3,329","11,513","3,103",Upgrade
6,Operating Expenses,"20,107","19,022","19,178","18,749","19,191","17,888","17,063","17,667","18,725",...,"14,500","14,883","15,409","15,798","15,431","14,636","15,552","23,626","15,866",Upgrade
7,Operating Income,"18,242","15,525","13,538","11,966","11,526","11,369","12,584","12,812","13,541",...,"9,312","9,183","7,341","8,671","9,247","8,579","8,365",-509,"9,345",Upgrade
8,Other Expense / Income,"2,704","2,699","2,019","1,555","1,900",-816,"-1,069","-1,833","-3,707",...,"1,449","1,421","1,282","1,194","1,104","1,226",444,-273,442,Upgrade
9,Pretax Income,"15,538","12,826","11,519","10,411","9,626","12,185","13,653","14,645","17,248",...,"7,863","7,762","6,059","7,477","8,143","7,353","7,921",-236,"8,903",Upgrade
10,Income Tax,"3,345","2,229","2,264","2,216","1,781","2,258","2,424","3,149","3,397",...,"2,087","2,310","1,570","2,349","2,575","2,460","3,020",414,"2,802",Upgrade


In [13]:
# Cash flow statement
pd.DataFrame(data["cash_flow_statement"]["quarterly"])

,Metrics,2023-03-31,2022-12-31,2022-09-30,2022-06-30,2022-03-31,2021-12-31,2021-09-30,2021-06-30,2021-03-31,...,2015-06-30,2015-03-31,2014-12-31,2014-09-30,2014-06-30,2014-03-31,2013-12-31,2013-09-30,2013-06-30,+69 Quarters
1,Net Income,"12,193","10,597","9,255","8,195","7,845","9,927","11,229","11,496","13,851",...,"5,776","5,452","4,489","5,128","5,568","4,893","4,901",-650,"6,101",Upgrade
2,Depreciation & Amortization,"1,649","1,671","1,771","1,789","1,820","1,931","1,928","2,003","2,070",...,"1,238","1,181","1,186","1,147","1,218","1,208","1,246","1,651","1,435",Upgrade
3,Share-Based Compensation,0,0,0,0,0,0,0,0,0,...,0,0,"-1,681",539,524,618,"-1,734",507,586,Upgrade
4,Other Operating Activities,"-125,083","88,954","-29,230","56,034","-51,582","73,237","10,174",31,"-59,793",...,"10,282","8,246","24,752","-9,263","-11,681","7,948","-11,327","24,875","60,398",Upgrade
5,Operating Cash Flow,"-111,241","101,222","-18,204","66,018","-41,917","85,095","23,331","13,530","-43,872",...,"17,296","14,879","28,746","-2,449","-4,371","14,667","-6,914","26,383","68,520",Upgrade
6,Operating Cash Flow Growth,-,18.95%,-,387.94%,-,-,-,-83.68%,-,...,-,1.45%,-,-,-,-26.53%,-,-,63.21%,Upgrade
7,Acquisitions,0,0,0,0,0,0,0,0,0,...,0,0,-24,43,-19,0,62,-17,-8,Upgrade
8,Change in Investments,"30,375","-47,101","42,272","-51,647","-69,411","-96,465","-58,430","20,551","16,044",...,"101,542","-26,094","-72,202","1,878","-29,663","-67,830","36,963","-46,259","-85,850",Upgrade
9,Other Investing Activities,"-6,581","-4,429","-2,750","-1,556","-3,197","-3,693","-3,845","-2,853",-653,...,79,"1,944","2,220",387,154,-580,"1,575",-580,-932,Upgrade
10,Investing Cash Flow,"23,794","-51,530","39,522","-53,203","-72,608","-100,158","-62,275","17,698","15,391",...,"101,621","-24,150","-70,006","2,308","-29,528","-68,410","38,600","-46,856","-86,790",Upgrade


In [14]:
# Ratios
pd.DataFrame(data["ratios"]["quarterly"])

,Metrics,Current,2023-03-31,2022-12-31,2022-09-30,2022-06-30,2022-03-31,2021-12-31,2021-09-30,2021-06-30,...,2015-06-30,2015-03-31,2014-12-31,2014-09-30,2014-06-30,2014-03-31,2013-12-31,2013-09-30,2013-06-30,+70 Quarters
1,Market Capitalization,"398,688","383,549","393,343","306,454","330,741","402,527","467,966","489,131","470,840",...,"251,463","225,861","233,936","226,580","218,075","229,898","219,837","194,571","199,529",Upgrade
2,Market Cap Growth,-,-4.71%,-15.95%,-37.35%,-29.75%,-13.35%,20.82%,66.72%,64.28%,...,15.31%,-1.76%,6.41%,16.45%,9.29%,26.56%,31.53%,26.53%,46.70%,Upgrade
3,Enterprise Value,"430,648","421,675","368,614","237,545","230,834","222,678","276,071","333,040","362,720",...,"368,450","258,533","286,674","346,215","381,650","415,776","398,909","358,739","471,187",Upgrade
4,PE Ratio,9.91,9.53,10.96,8.70,8.89,9.94,10.06,10.13,10.22,...,12.06,10.94,11.65,11.06,14.82,15.08,13.28,11.51,8.71,Upgrade
5,PS Ratio,2.88,2.81,3.06,2.48,2.75,3.35,3.85,4.02,3.88,...,2.64,2.35,2.46,2.35,2.30,2.41,2.26,2.00,2.01,Upgrade
6,PB Ratio,1.30,1.26,1.35,1.06,1.16,1.41,1.59,1.69,1.64,...,1.04,0.96,1.01,0.98,0.96,1.05,1.04,0.94,0.95,Upgrade
7,Debt / Equity Ratio,1.93,1.93,1.86,2.00,1.99,2.01,1.87,2.08,2.09,...,2.24,2.38,2.44,2.42,2.57,2.66,2.53,2.74,2.93,Upgrade
8,Dividend Yield,2.93%,0.78%,0.75%,0.97%,0.90%,0.74%,0.64%,0.55%,0.58%,...,0.60%,0.67%,0.64%,0.67%,0.66%,0.63%,0.66%,0.74%,0.57%,Upgrade
9,Payout Ratio,29.50%,24.30%,28.00%,31.90%,36.10%,37.90%,29.90%,24.10%,23.70%,...,25.60%,27.40%,33.30%,29.20%,25.90%,29.50%,29.20%,-223.50%,18.60%,Upgrade
10,Buyback Yield / Dilution,0.28%,0.28%,0.47%,1.32%,2.49%,3.18%,3.37%,2.52%,1.27%,...,1.81%,1.73%,0.84%,-0.58%,0.05%,0.61%,0.63%,1.23%,0.16%,Upgrade


In [17]:
pd.DataFrame(data["balance_sheet"]["quarterly"])

,Metrics,2023-03-31,2022-12-31,2022-09-30,2022-06-30,2022-03-31,2021-12-31,2021-09-30,2021-06-30,2021-03-31,...,2015-06-30,2015-03-31,2014-12-31,2014-09-30,2014-06-30,2014-03-31,2013-12-31,2013-09-30,2013-06-30,+70 Quarters
1,Cash & Equivalents,"546,000","567,234","644,187","669,260","754,532","740,834","759,869","705,421","711,072",...,"422,902","529,204","512,308","439,684","421,432","398,852","355,822","402,109","340,532",Upgrade
2,Cash & Cash Equivalents,"546,000","567,234","644,187","669,260","754,532","740,834","759,869","705,421","711,072",...,"422,902","529,204","512,308","439,684","421,432","398,852","355,822","402,109","340,532",Upgrade
3,Cash Growth,-27.64%,-23.43%,-15.22%,-5.13%,6.11%,40.41%,55.86%,42.88%,93.47%,...,0.35%,32.68%,43.98%,9.34%,23.76%,31.57%,102.70%,155.00%,94.31%,Upgrade
4,Receivables,"115,316","125,189","143,905","145,442","152,207","102,570","116,395","125,253","114,754",...,"69,642","70,006","70,079","75,504","77,096","73,122","65,160","66,269","81,562",Upgrade
5,"Property, Plant & Equipment","28,266","27,734","27,199","26,770","26,916","27,070","26,996","26,631","26,926",...,"15,073","14,963","15,133","15,177","15,216","14,919","14,891","14,876","14,574",Upgrade
6,Goodwill and Intangibles,"62,090","60,859","60,806","59,360","58,485","56,691","56,566","54,655","54,588",...,"48,567","48,581","48,839","49,244","49,449","49,554","49,699","49,917","50,008",Upgrade
7,Total Assets,"3,744,305","3,665,743","3,773,884","3,841,314","3,954,687","3,743,567","3,757,576","3,684,256","3,689,336",...,"2,449,599","2,577,148","2,572,274","2,527,005","2,520,336","2,476,986","2,415,689","2,463,309","2,439,494",Upgrade
8,Accounts Payable,"275,077","300,141","300,016","313,326","320,671","262,755","268,604","297,082","285,066",...,"191,749","202,157","206,939","211,055","203,885","202,499","194,491","212,283","211,432",Upgrade
9,Total Liabilities,"3,441,223","3,373,411","3,485,866","3,555,171","3,668,788","3,449,440","3,467,535","3,397,870","3,408,622",...,"2,208,394","2,341,284","2,340,547","2,295,728","2,293,022","2,257,331","2,204,511","2,256,639","2,230,255",Upgrade
10,Total Debt,"584,126","542,505","575,278","569,353","574,683","548,939","603,778","597,301","638,424",...,"539,889","561,876","565,046","559,319","585,007","584,730","534,894","566,277","612,190",Upgrade


### Stock price 

In [18]:
pd.DataFrame(data["stock_price"])

,symbol,date,open,high,low,close,volume,adjclose,dividends
0,JPM,2017-01-03,87.339996,87.760002,85.980003,87.230003,20550700,72.665390,0.00
1,JPM,2017-01-04,86.959999,87.180000,86.400002,86.910004,15266600,72.799416,0.48
2,JPM,2017-01-05,86.809998,87.110001,85.260002,86.110001,14300800,72.129318,0.00
3,JPM,2017-01-06,86.389999,86.620003,85.940002,86.120003,12893300,72.137703,0.00
4,JPM,2017-01-09,85.730003,86.769997,85.519997,86.180000,12806600,72.187965,0.00
...,...,...,...,...,...,...,...,...,...
1315,JPM,2022-03-24,140.300003,140.710007,139.110001,140.690002,10017600,135.136185,0.00
1316,JPM,2022-03-25,141.089996,143.179993,140.800003,141.919998,8383500,136.317627,0.00
1317,JPM,2022-03-28,140.000000,140.970001,137.899994,140.869995,10818100,135.309082,0.00
1318,JPM,2022-03-29,143.350006,143.600006,140.240005,141.179993,11316500,135.606827,0.00
